In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor 
from sklearn.tree import DecisionTreeClassifier 
from math import sqrt
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import BaggingClassifier, BaggingRegressor, GradientBoostingClassifier, GradientBoostingRegressor, StackingClassifier
warnings.filterwarnings('ignore')
from catboost import CatBoostRegressor, CatBoostClassifier

In [2]:
data_regress= pd.read_csv("../data/trip_duration_task_m.csv")
data_regress.drop(["Unnamed: 0"], axis=1, inplace=True)
data_regress

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
0,-73.953918,40.778873,-73.963875,40.771164,400
1,-73.988312,40.731743,-73.994751,40.694931,1100
2,-73.997314,40.721458,-73.948029,40.774918,1635
3,-73.961670,40.759720,-73.956779,40.780628,1141
4,-74.017120,40.708469,-73.988182,40.740631,848
...,...,...,...,...,...
199489,-73.978088,40.751461,-73.964417,40.764450,565
199490,-73.988548,40.721390,-73.998604,40.693054,800
199491,-73.987770,40.732391,-73.971451,40.760799,472
199492,-73.870796,40.773720,-73.988571,40.721371,2163


In [3]:
y = data_regress['trip_duration']
X = data_regress.drop(['trip_duration'], axis=1)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

In [6]:
def DecisionTreeRegressorOptimal(x_train, y_train, x_test, y_test):
    parameters={"splitter":["best","random"],
            "max_depth" : [1,3,5,7,9,11,12],
           "min_samples_leaf":[1,2,3,4,5,6,7,8,9,10],
           "min_weight_fraction_leaf":[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
           "max_features":["auto","log2","sqrt",None],
           "max_leaf_nodes":[None,10,20,30,40,50,60,70,80,90] }
    search = RandomizedSearchCV(DecisionTreeRegressor(),param_distributions=parameters, scoring="neg_mean_squared_error", )
    
    optimalDecitionTreeReg = search.fit(x_train,y_train)
    y_pred = optimalDecitionTreeReg.predict(x_test)
    print(search.best_params_)
    print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
    print(f'MSE: {mean_squared_error(y_test, y_pred)}')
    print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
    print(f'MAPE: {(mean_absolute_percentage_error(y_test, y_pred))}')
    print(f'R^2: {r2_score(y_test, y_pred)}')

In [7]:
DecisionTreeRegressorOptimal(X_train,y_train, X_test, y_test)

{'splitter': 'best', 'min_weight_fraction_leaf': 0.4, 'min_samples_leaf': 4, 'max_leaf_nodes': 50, 'max_features': None, 'max_depth': 11}
MAE: 620.8347769605313
MSE: 9975731.263952762
RMSE: 3158.438105132466
MAPE: 1.5549953163501835
R^2: 0.00033608365039894394


In [8]:
data_class = pd.read_csv("../data/csgo_task_m.csv")
data_class.drop(["Unnamed: 0"], axis=1, inplace=True)
data_class.head()

,time_left,ct_score,t_score,bomb_planted,ct_health,t_health,ct_money,t_money,ct_players_alive,t_players_alive,map_de_cache,map_de_dust2,map_de_inferno,map_de_mirage,map_de_nuke,map_de_overpass,map_de_train,map_de_vertigo
0,175.00,0.0,0.0,1,500.0,500.0,4000.0,4000.0,5.0,5.0,0,1,0,0,0,0,0,0
1,156.03,0.0,0.0,1,500.0,500.0,600.0,650.0,5.0,5.0,0,1,0,0,0,0,0,0
2,96.03,0.0,0.0,1,391.0,400.0,750.0,500.0,4.0,4.0,0,1,0,0,0,0,0,0
3,76.03,0.0,0.0,1,391.0,400.0,750.0,500.0,4.0,4.0,0,1,0,0,0,0,0,0
4,174.97,1.0,0.0,1,500.0,500.0,18350.0,10750.0,5.0,5.0,0,1,0,0,0,0,0,0


In [9]:
X = data_class.values
y = data_class["bomb_planted"].values

In [10]:
overSampler = RandomOverSampler()
X_over_sample, y_over_sample = overSampler.fit_resample(X,y)

In [11]:
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_over_sample, y_over_sample, test_size=0.2, random_state=4)

In [12]:
def DecisionTreeClassifierOptimal(x_train, y_train, x_test, y_test):
    parameters={"splitter":["best","random"],
            "max_depth" : [1,3,5,7,9,11,12],
           "min_samples_leaf":[1,2,3,4,5,6,7,8,9,10],
           "min_weight_fraction_leaf":[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
           "max_features":["auto","log2","sqrt",None],
           "max_leaf_nodes":[None,10,20,30,40,50,60,70,80,90] }
    search = RandomizedSearchCV(DecisionTreeClassifier(),param_distributions=parameters, scoring="neg_mean_squared_error", )
    optimalDecitionTreeClass = search.fit(x_train,y_train)
    predictions = optimalDecitionTreeClass.predict(x_test)
    print(search.best_params_)
    print(accuracy_score(y_test, predictions))
    print(confusion_matrix(y_test, predictions))
    print(precision_score(y_test, predictions))
    print(recall_score(y_test, predictions))
    print(f1_score(y_test, predictions))
    

In [13]:
DecisionTreeClassifierOptimal(X_train_c,y_train_c, X_test_c, y_test_c)

{'splitter': 'best', 'min_weight_fraction_leaf': 0.1, 'min_samples_leaf': 7, 'max_leaf_nodes': 90, 'max_features': None, 'max_depth': 12}
1.0
[[19769     0]
 [    0 19937]]
1.0
1.0
1.0


In [14]:
def bagClassOptimal(X_train, y_train, X_test, y_test ):
    bag = BaggingClassifier()
    n_estimators = [20, 30]
    max_samples = [5, 10, 25]
    max_features = [1, 2, 5]
    hyperbag = dict(n_estimators = n_estimators, max_samples = max_samples, 
                max_features = max_features, )
    gridbag = RandomizedSearchCV(bag, hyperbag)
    bestbag = gridbag.fit(X_train, y_train)
    predictions = bestbag.predict(X_test)
    print(bestbag.best_params_)
    print(accuracy_score(y_test, predictions))
    print(confusion_matrix(y_test, predictions))
    print(precision_score(y_test, predictions))
    print(recall_score(y_test, predictions))
    print(f1_score(y_test, predictions))
    

In [15]:
bagClassOptimal(X_train_c,y_train_c, X_test_c, y_test_c)

{'n_estimators': 30, 'max_samples': 25, 'max_features': 5}
0.9925955774945852
[[19726    43]
 [  251 19686]]
0.9978204673323534
0.9874103425791242
0.99258811072455


In [16]:
def bagRegOptimal(X_train, y_train, X_test, y_test ):
    bag = BaggingRegressor()
    n_estimators = [20, 30]
    max_samples = [5, 10, 25]
    max_features = [1, 2, 5]
    hyperbag = dict(n_estimators = n_estimators, max_samples = max_samples, 
                max_features = max_features)
    gridbag = RandomizedSearchCV(bag, hyperbag)
    bestbag = gridbag.fit(X_train, y_train)
    y_pred = bestbag.predict(X_test)
    print(bestbag.best_params_)
    print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
    print(f'MSE: {mean_squared_error(y_test, y_pred)}')
    print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
    print(f'MAPE: {(mean_absolute_percentage_error(y_test, y_pred))}')
    print(f'R^2: {r2_score(y_test, y_pred)}')

In [17]:
bagRegOptimal(X_train,y_train, X_test, y_test)


{'n_estimators': 20, 'max_samples': 25, 'max_features': 1}
MAE: 669.6860284718915
MSE: 10479222.843758803
RMSE: 3237.162776840053
MAPE: 1.6206225805901564
R^2: -0.05011859994123968


In [24]:
def gradientBoostingClassOptimal(X_train, y_train, X_test, y_test ):
    grad = GradientBoostingClassifier()
    n_estimators = [20, 30]
    max_depth = np.arange(1, 10, 2)
    learning_rate = np.arange(0,1,0.1)
    hyperGrad = dict(n_estimators = n_estimators, learning_rate = learning_rate, max_depth = max_depth)
    grid = RandomizedSearchCV(grad, hyperGrad)
    bestmodel = grid.fit(X_train, y_train)
    predictions = bestmodel.predict(X_test)
    print(bestmodel.best_params_)
    print(accuracy_score(y_test, predictions))
    print(confusion_matrix(y_test, predictions))
    print(precision_score(y_test, predictions))
    print(recall_score(y_test, predictions))
    print(f1_score(y_test, predictions))

In [25]:
gradientBoostingClassOptimal(X_train_c,y_train_c, X_test_c, y_test_c)

{'n_estimators': 30, 'max_depth': 9, 'learning_rate': 0.30000000000000004}
1.0
[[19769     0]
 [    0 19937]]
1.0
1.0
1.0


In [35]:
def gradientBoostingRegOptimal(X_train, y_train, X_test, y_test ):
    grad = GradientBoostingRegressor()
    n_estimators = [20, 30]
    max_depth = np.arange(1, 7, 2)
    learning_rate = np.arange(0,1,0.2)
    hyperGrad = dict(n_estimators = n_estimators, learning_rate = learning_rate, max_depth = max_depth)
    grid = RandomizedSearchCV(grad, hyperGrad)
    bestmodel = grid.fit(X_train, y_train)
    y_pred = bestmodel.predict(X_test)
    print(bestmodel.best_params_)
    print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
    print(f'MSE: {mean_squared_error(y_test, y_pred)}')
    print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
    print(f'MAPE: {(mean_absolute_percentage_error(y_test, y_pred))}')
    print(f'R^2: {r2_score(y_test, y_pred)}')

In [27]:
gradientBoostingRegOptimal(X_train,y_train, X_test, y_test)

{'n_estimators': 20, 'max_depth': 1, 'learning_rate': 0.5}
MAE: 559.7741544713288
MSE: 9875198.799008347
RMSE: 3142.4829035347743
MAPE: 1.6205853814549738
R^2: 0.010410400506724482


In [32]:
def stackClassifier(X_train, y_train, X_test, y_test):
    estimators = [
     ('knn', KNeighborsClassifier(n_neighbors=5)),
     ('rfc', RandomForestClassifier())]
    params = {'knn__n_neighbors': [3,5,9],
         'rfc__n_estimators': [20, 30],
         'rfc__max_depth': np.arange(1, 10, 2),
         'final_estimator__C': np.arange(1, 10, 2)} 
    clf = StackingClassifier(
     estimators=estimators, final_estimator=LogisticRegression())
    grid = RandomizedSearchCV(estimator=clf, param_distributions=params, scoring='accuracy', )
    bestmodel = grid.fit(X_train, y_train)
    predictions = bestmodel.predict(X_test)
    print(bestmodel.best_params_)
    print(accuracy_score(y_test, predictions))
    print(confusion_matrix(y_test, predictions))
    print(precision_score(y_test, predictions))
    print(recall_score(y_test, predictions))
    print(f1_score(y_test, predictions))

In [33]:
stackClassifier(X_train_c,y_train_c, X_test_c, y_test_c)

{'rfc__n_estimators': 30, 'rfc__max_depth': 9, 'knn__n_neighbors': 3, 'final_estimator__C': 1}
1.0
[[19769     0]
 [    0 19937]]
1.0
1.0
1.0


In [39]:
def stackRegressor(X_train, y_train, X_test, y_test):
    estimators = [
    ('lr', Ridge()),
    ('svr', LinearSVR())]
    parameters = {'lr__alpha': np.arange(0, 1, 0.1),
                  'svr__C': np.arange(1, 6, 2),
                  'final_estimator__max_depth': [3,5]
                  }

    stackinReg = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor()
)
    grid = RandomizedSearchCV(estimator=stackinReg, param_distributions=parameters )
    bestmodel = grid.fit(X_train, y_train)
    y_pred = bestmodel.predict(X_test)
    print(bestmodel.best_params_)
    print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
    print(f'MSE: {mean_squared_error(y_test, y_pred)}')
    print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
    print(f'MAPE: {(mean_absolute_percentage_error(y_test, y_pred))}')
    print(f'R^2: {r2_score(y_test, y_pred)}')

In [40]:
stackRegressor(X_train, y_train, X_test, y_test)

{'svr__C': 1, 'lr__alpha': 0.7000000000000001, 'final_estimator__max_depth': 5}
MAE: 539.1982426438315
MSE: 9851410.898266718
RMSE: 3138.6957320305387
MAPE: 1.4018230454830398
R^2: 0.01279417623082002


In [41]:
def CatBoostClassifierOptimal(x_train, y_train, x_test, y_test):
    parameters={
            "depth" : [1,3],
           "learning_rate": np.arange(0, 1, 0.5,) }
    search = RandomizedSearchCV(CatBoostClassifier(logging_level="Silent"),param_distributions=parameters )
    optimalDecitionTreeClass = search.fit(x_train,y_train)
    predictions = optimalDecitionTreeClass.predict(x_test)
    print(search.best_params_)
    print(accuracy_score(y_test, predictions))
    print(confusion_matrix(y_test, predictions))
    print(precision_score(y_test, predictions))
    print(recall_score(y_test, predictions))
    print(f1_score(y_test, predictions))

In [42]:
CatBoostClassifierOptimal(X_train_c, y_train_c, X_test_c, y_test_c)

{'learning_rate': 0.5, 'depth': 1}
1.0
[[19769     0]
 [    0 19937]]
1.0
1.0
1.0


In [43]:
def CatBoostRegressorOptima(x_train, y_train, x_test, y_test):
    parameters={
            "depth" : [1,3],
           "learning_rate": np.arange(0, 1, 0.5,) }
    search = RandomizedSearchCV(CatBoostRegressor(logging_level="Silent"),param_distributions=parameters )
    
    optimalDecitionTreeReg = search.fit(x_train,y_train)
    y_pred = optimalDecitionTreeReg.predict(x_test)
    print(search.best_params_)
    print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
    print(f'MSE: {mean_squared_error(y_test, y_pred)}')
    print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
    print(f'MAPE: {(mean_absolute_percentage_error(y_test, y_pred))}')
    print(f'R^2: {r2_score(y_test, y_pred)}')

In [44]:
CatBoostRegressorOptima(X_train,y_train, X_test, y_test)

{'learning_rate': 0.5, 'depth': 1}
MAE: 554.27981650326
MSE: 9941325.852432301
RMSE: 3152.9868145034006
MAPE: 1.5947697198870405
R^2: 0.0037838357513847454
